In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/taxi-trips-2023/Taxi_Trips_-_2023.csv
/kaggle/input/taxi-trips-2022-v-2/Taxi_Trips_-_2022.csv


In [2]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285413 sha256=8c3a44d404efe1d873a2c3266302ccbcf20be169883ff60dc2145b326999dd8c
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [181]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
import seaborn as sns

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import StringType, ShortType, IntegerType, TimestampType, LongType, StructType, StructField, DoubleType
from pyspark.ml.evaluation import RegressionEvaluator

from pandas.tseries.holiday import USFederalHolidayCalendar

from statsmodels.tsa.seasonal import seasonal_decompose, DecomposeResult

from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.feature import VectorAssembler
from xgboost.spark import SparkXGBRegressor
from prophet import Prophet

from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

In [4]:
#https://xgboost.readthedocs.io/en/stable/tutorials/spark_estimator.html#xgboost-pyspark-gpu-support

# python -m venv xgboost_env
# source xgboost_env/bin/activate
# pip install pyarrow pandas venv-pack xgboost
# # https://docs.rapids.ai/install#pip-install
# pip install cudf-cu11 --extra-index-url=https://pypi.nvidia.com
# venv-pack -o xgboost_env.tar.gz

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.plotting.backend = "plotly"

## Описание данных

- Trip ID - A unique identifier for the trip.
- Taxi ID - A unique identifier for the taxi.
- Trip Start Timestamp - When the trip started, rounded to the nearest 15 minutes.
- Trip End Timestamp - When the trip ended, rounded to the nearest 15 minutes.
- Trip Seconds - Time of the trip in seconds.
- Trip Miles - Distance of the trip in miles.
- Pickup Census Tract - The Census Tract where the trip began. For privacy, this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.
- Dropoff Census Tract - The Census Tract where the trip ended. For privacy, this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.
- Pickup Community Area - The Community Area where the trip began. This column will be blank for locations outside Chicago.
- Dropoff Community Area -The Community Area where the trip ended. This column will be blank for locations outside Chicago.
- Fare - The fare for the trip.
- Tips -The tip for the trip. Cash tips generally will not be recorded.
- Tolls -The tolls for the trip.
- Extras - Extra charges for the trip.
- Trip Total - Total cost of the trip, the total of the previous columns.
- Payment Type - Type of payment for the trip.
- Company -The taxi company.

# 1. Загружаем и преобразуем данные

**Источник данных**

[][https://data.cityofchicago.org/Transportation/Taxi-Trips-2022/npd7-ywjz]

[][https://data.cityofchicago.org/Transportation/Taxi-Trips-2023/e55j-2ewb]

In [7]:
spark = SparkSession.builder.appName('taxi').getOrCreate()
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/01 14:15:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
print('Spark Version: {}'.format(spark.version))

Spark Version: 3.4.1


In [9]:
file_path_1 = '/kaggle/input/taxi-trips-2022-v-2/Taxi_Trips_-_2022.csv'
file_path_2 = '/kaggle/input/taxi-trips-2023/Taxi_Trips_-_2023.csv'

df_1 = spark.read.csv(file_path_1,
                    header = True,
                    inferSchema = True,
                    nanValue = '')

# df_2 = spark.read.csv(file_path_2,
#                     header = True,
#                     inferSchema = True,
#                     nanValue = '')

In [10]:
df = df_1

**Размер датафрейма**

In [11]:
(df.count(), len(df.columns))

(6382425, 23)

## 1.1. Schema

In [12]:
df.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

## 1.2. Изменяем наименование колонок

In [13]:
df = df.toDF(*['_'.join(i.lower().split()) for i in df.columns])

In [14]:
df.columns

['trip_id',
 'taxi_id',
 'trip_start_timestamp',
 'trip_end_timestamp',
 'trip_seconds',
 'trip_miles',
 'pickup_census_tract',
 'dropoff_census_tract',
 'pickup_community_area',
 'dropoff_community_area',
 'fare',
 'tips',
 'tolls',
 'extras',
 'trip_total',
 'payment_type',
 'company',
 'pickup_centroid_latitude',
 'pickup_centroid_longitude',
 'pickup_centroid_location',
 'dropoff_centroid_latitude',
 'dropoff_centroid_longitude',
 'dropoff_centroid_location']

## 1.2. Проверяем на пропуски

In [15]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

## 1.3. Изменяем тип данных

In [16]:
df = df.withColumn('trip_start_timestamp', F.to_timestamp(F.col('trip_start_timestamp'), 'MM/dd/yyyy hh:mm:ss a'))
df = df.withColumn('trip_end_timestamp', F.to_timestamp(F.col('trip_end_timestamp'), 'MM/dd/yyyy hh:mm:ss a'))
df = df.withColumn("pickup_community_area", F.col("pickup_community_area").cast("int"))
df = df.withColumn("dropoff_community_area",F. col("dropoff_community_area").cast("int"))

## 1.4. Создаем дополнительные фитчи

In [17]:
df.limit(5).toPandas()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,bcfa19f2539021c054809d4c3993d226996ae095,368ce5511598af2cc07efdb68067d381174fdac1d47a28...,2022-01-01,2022-01-01 00:00:00,152,0.100,NaN,NaN,NaN,NaN,3.750,0.000,0.000,0.000,3.750,Cash,Medallion Leasin,NaN,NaN,None,NaN,NaN,None
1,2aba69ff015f9ea8e7bff43cab7eddb228f34a12,449fa4909552757130d09d98ebc7770e2dd94579036b0e...,2022-01-01,2022-01-01 00:30:00,2360,17.440,NaN,NaN,NaN,8.000,47.750,0.000,0.000,5.000,52.750,Cash,Flash Cab,NaN,NaN,None,41.900,-87.633,POINT (-87.6333080367 41.899602111)
2,54d812a0b88f8f9707825261014b3563a0a60ace,f98ae5e71fdda8806710af321dce58002146886c013f41...,2022-01-01,2022-01-01 00:00:00,536,4.830,NaN,NaN,28.000,22.000,14.750,0.000,0.000,0.000,14.750,Cash,Globe Taxi,41.874,-87.664,POINT (-87.6635175498 41.874005383),41.923,-87.699,POINT (-87.6991553432 41.9227606205)
3,7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef,8eca35a570101ad24c638f1f43eecce9d0cb7843e13a75...,2022-01-01,2022-01-01 00:15:00,897,2.070,NaN,NaN,8.000,32.000,9.750,0.000,0.000,1.500,11.250,Cash,Sun Taxi,41.900,-87.633,POINT (-87.6333080367 41.899602111),41.879,-87.625,POINT (-87.6251921424 41.8788655841)
4,f1a650ee419b4e52d766432e6f86eca3457bfb37,e2d8418fcdb061eee0a4318fba0a6a1200aaff0143feb0...,2022-01-01,2022-01-01 00:30:00,2200,2.480,NaN,NaN,8.000,32.000,9.360,2.140,0.000,0.000,11.500,Mobile,Chicago Independents,41.900,-87.633,POINT (-87.6333080367 41.899602111),41.879,-87.625,POINT (-87.6251921424 41.8788655841)


**Минимальная дата и максимальная дата**

In [18]:
t = df.agg(F.min('trip_start_timestamp').alias('min_date'), F.max('trip_start_timestamp').alias('max_date'))
t.show()

+-------------------+-------------------+
|           min_date|           max_date|
+-------------------+-------------------+
|2022-01-01 00:00:00|2022-12-31 23:45:00|
+-------------------+-------------------+



In [19]:
min_start_date = t.head()[0]
max_start_date = t.head()[1]

In [20]:
min_start_date, max_start_date

(datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 12, 31, 23, 45))

## 1.4. Drop unnesassery columns

In [21]:
columns_to_drop = [ 'trip_id', 'taxi_id', 'pickup_centroid_latitude', 'pickup_centroid_longitude',
                   'pickup_centroid_longitude', 'pickup_centroid_longitude', 'pickup_centroid_longitude',
                   'pickup_centroid_longitude', 'dropoff_centroid_longitude', 'dropoff_centroid_location',
                   'pickup_centroid_location', 'pickup_centroid_location', 'dropoff_centroid_latitude',
                    'pickup_census_tract', 'dropoff_census_tract', 'company', 'payment_type', 'fare', 'tips',
                   'tolls', 'extras','trip_total']
df = df.drop(*columns_to_drop)

In [22]:
#look
df.show(10)

+--------------------+-------------------+------------+----------+---------------------+----------------------+
|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles|pickup_community_area|dropoff_community_area|
+--------------------+-------------------+------------+----------+---------------------+----------------------+
| 2022-01-01 00:00:00|2022-01-01 00:00:00|         152|       0.1|                 null|                  null|
| 2022-01-01 00:00:00|2022-01-01 00:30:00|        2360|     17.44|                 null|                     8|
| 2022-01-01 00:00:00|2022-01-01 00:00:00|         536|      4.83|                   28|                    22|
| 2022-01-01 00:00:00|2022-01-01 00:15:00|         897|      2.07|                    8|                    32|
| 2022-01-01 00:00:00|2022-01-01 00:30:00|        2200|      2.48|                    8|                    32|
| 2022-01-01 00:00:00|2022-01-01 00:15:00|        1256|     13.29|                   76|                

## 1.5. Заполняем пропуски

In [23]:
df = df.na.fill(99,subset=["pickup_community_area"])
df = df.na.fill(98,subset=["dropoff_community_area"])

### 1.6. Посмотрим на количество районов

In [24]:
# df.select(F.countDistinct("pickup_community_area")).show()

In [25]:
# df.select("pickup_community_area").distinct().show()

## 1.6. Анализ целевой переменной

### 1.6.1 Количество заказов по районам

In [26]:
# t = df.groupBy("pickup_community_area").count().toPandas()
# t = t.sort_values(by='count',ascending=False)
# t['pickup_community_area'] = t['pickup_community_area'].astype('str')


# fig = px.bar(t, y='pickup_community_area', x='count',
#               title='Количество заказов по районам',
#               labels={
#                      "pickup_community_area": "Район",
#                      "count": "Количество"
#                  },
#               width=1200,
#               height=700,
#               orientation='h')
# fig.update_layout(yaxis=dict(autorange="reversed"))
# fig.show()

По количеству заказов разброс очень большой. Есть районы с большим количество есть с маленьким.

### 1.6.1 Количество заказов по дню по всем районам[](http://)

In [27]:
# t = df.groupby(F.date_trunc('day', F.col('trip_start_timestamp')).alias('start_date')) \
#                                                      .agg(F.count('*').alias('cnt_target')) \
#                                                      .orderBy('start_date') \
#                                                      .toPandas()

# t = t.set_index('start_date')


# fig = px.line(t, t.index, y="cnt_target",
#               title='Количество заказов',
#               labels={
#                      "start_date_hour": "Дата час",
#                      "cnt_target": "Количество"
#                  },
#               width=1400,
#               height=600)
# fig.show()

Видим тренд к лету увеличивается количестов заказов, а также сезонность по неделям

### 1.6.3 Количество заказов по районам

In [28]:
# t = df.groupBy(F.date_trunc('day', F.col('trip_start_timestamp')).alias('date'))\
#                                                                  .agg(F.count('*').alias('cnt_target')) \
#                                                                  .orderBy('date') \
#                                                                  .toPandas()

# t = t.set_index('date')

# fig = px.line(t, t.index, y="cnt_target",
#               title='Количество заказов по дню недели',
#               labels={
#                      "hour_trip_start_timestamp": "Дата",
#                      "cnt_target": "Количество"
#                  },
#               width=1400,
#               height=600)
# fig.show()

### 1.4.3 По районам

In [29]:
# t = df.groupBy('pickup_community_area', F.date_trunc('day', F.col('trip_start_timestamp')).alias('date'))\
#                                                                  .agg(F.count('*').alias('count')) \
#                                                                  .orderBy('date') \
#                                                                  .toPandas()

# t = t.set_index('date')

# fig = px.line(t, t.index, y="count",
#               title='Количество заказов по дню недели',
#               labels={
#                      "hour_trip_start_timestamp": "Дата",
#                      "count": "Количество"
#                  },
#               color = 'pickup_community_area',
#               width=1400,
#               height=600)
# fig.show()

Распределение по районам приблизительно одинаковые

Посмотрим на маленькие районы по которым мало данных:

In [30]:
# s = set([11, 53, 5, 2, 3])

# fig = px.line(t[t.pickup_community_area.isin(s)], t[t.pickup_community_area.isin(s)].index, y="count",
#               title='Количество заказов по дню недели',
#               labels={
#                      "hour_trip_start_timestamp": "Дата",
#                      "count": "Количество"
#                  },
#               color = 'pickup_community_area',
#               width=1400,
#               height=600)
# fig.show()

### 1.6.3. Количество заказов по месяцам по районам

In [31]:
# t = df.groupBy('pickup_community_area', F.month(F.col("trip_start_timestamp")).alias('month')) \
#                                                                      .agg(F.count('*').alias('count')) \
#                                                                      .toPandas().set_index('month') \
#                                                                      .sort_values(by=['month', 'count'],ascending=[True, True])

# fig = px.bar(t, y=t.index, x='count',
#               title='Количество заказов по месяцам',
#               labels={
#                      "count": "Количество",
#                      "month": "Месяц"
#                  },
#               width=1200,
#               height=600,
#               color='pickup_community_area',
#               orientation='h')
# fig.update_layout(yaxis=dict(autorange="reversed"))
# fig.show()

## 1.6.1 Недельная сезонность

In [32]:
# t = df.groupBy(F.date_trunc('day', F.col('trip_start_timestamp')).alias('date'))\
#                                                                  .agg(F.count('*').alias('count')) \
#                                                                  .orderBy('date') \
#                                                                  .toPandas()

# t = t.set_index('date')

In [33]:
# t = df.filter(F.month(F.col("trip_start_timestamp")) == 4).groupBy(F.date_trunc('day', F.col('trip_start_timestamp'))
#                                                                  .alias('date'))\
#                                                                  .agg(F.count('*').alias('count')) \
#                                                                  .orderBy('date') \
#                                                                  .toPandas()

# t = t.set_index('date')

In [34]:
# fig = px.line(t, t.index, y="count",
#               title='Количество заказов в апреле',
#               labels={
#                      "date": "Дата",
#                      "count": "Количество"
#                  },
#               width=1400,
#               height=600)
# fig.show()

Видим еженедельную сезонность

In [35]:
# #функция отрисовки
# def plot_seasonal_decompose(result:DecomposeResult, dates:pd.Series=None, title:str="Seasonal Decomposition"):
#     x_values = dates if dates is not None else np.arange(len(result.observed))
#     return (
#         make_subplots(
#             rows=4,
#             cols=1,
#             subplot_titles=["Observed", "Trend", "Seasonal", "Residuals"],
#         )
#         .add_trace(
#             go.Scatter(x=x_values, y=result.observed, mode="lines", name='Observed'),
#             row=1,
#             col=1,
#         )
#         .add_trace(
#             go.Scatter(x=x_values, y=result.trend, mode="lines", name='Trend'),
#             row=2,
#             col=1,
#         )
#         .add_trace(
#             go.Scatter(x=x_values, y=result.seasonal, mode="lines", name='Seasonal'),
#             row=3,
#             col=1,
#         )
#         .add_trace(
#             go.Scatter(x=x_values, y=result.resid, mode="lines", name='Residual'),
#             row=4,
#             col=1,
#         )
#         .update_layout(
#             height=900, title=f'<b>{title}</b>', margin={'t':100}, title_x=0.5, showlegend=False
#         )
#     )

In [36]:
# decomposition = seasonal_decompose(t['count'], model='additive')
# fig = plot_seasonal_decompose(decomposition, dates=t.index)
# fig.show()

### 1.6.1 Количество заказов в течение недели

In [37]:
# t = df.filter((F.month(F.col("trip_start_timestamp")) == 3) & (F.date_format(F.col("trip_start_timestamp"), "W")==2)) \
#                                 .groupBy(F.date_trunc('hour', F.col('trip_start_timestamp')).alias('hour')) \
#                                 .agg(F.count('*').alias('count')) \
#                                 .orderBy('hour') \
#                                 .toPandas().set_index('hour')

In [38]:
# fig = px.line(t, t.index, y="count",
#               title='Количество заказов по дню недели',
#               labels={
#                      "hour_trip_start_timestamp": "Дата",
#                      "count": "Количество"
#                  },
#               width=1400,
#               height=600)
# fig.show()

In [39]:
# decomposition = seasonal_decompose(t['count'], model='additive')
# fig = plot_seasonal_decompose(decomposition, dates=t.index)
# fig.show()

Есть дневная сезонность и тренд внутри недели на повышение и тренды внутри дня

### Количество заказов в течение месяца

In [40]:
# df.groupby('trip_start_timestamp', F.window('trip_start_timestamp','1 day')) \
#                                 .agg(F.count('*').alias('count')) \
#                                 .orderBy('window') \
#                                 .show(10)

In [41]:
# t = df.groupby(F.window('trip_start_timestamp','1 day')) \
#                                 .agg(F.count('*').alias('count')) \
#                                 .orderBy('window') \
#                                 .toPandas()

In [42]:
# t.head(10)

In [43]:
# fig = px.line(t, t.index, y="count",
#               title='Количество заказов по дню недели',
#               labels={
#                      "hour_trip_start_timestamp": "Дата",
#                      "count": "Количество"
#                  },
#               width=1400,
#               height=600)
# fig.show()

## 2.5 !!! Выводы по EDA

1. Есть недельная сезонность
2. Есть дневная сезонность
3. !!! Добавить праздники US Holiday
5. 

# 4. Feature Enginiring

In [44]:
AREA = 32

## 4.1. **Создаем Range из даты начала и окончания**

In [45]:
def generate_series(start, stop, interval):
    """
    :param start  - lower bound, inclusive
    :param stop   - upper bound, exclusive
    :interval int - increment interval in seconds
    
    """

    start, stop = spark.createDataFrame(
        [(start, stop)], ("start", "stop")
    ).select(
        [F.col(c).cast("timestamp").cast("long") for c in ("start", "stop")
    ]).first()
    return spark.range(start, stop, interval).select(
        F.col("id").cast("timestamp").alias("ds")
    )

In [46]:
min_start_date, max_start_date

(datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 12, 31, 23, 45))

In [47]:
generate_series(min_start_date, max_start_date, 60 * 60).show(5)
date_range_t = generate_series(min_start_date, max_start_date, 60 * 60)

+-------------------+
|                 ds|
+-------------------+
|2022-01-01 00:00:00|
|2022-01-01 01:00:00|
|2022-01-01 02:00:00|
|2022-01-01 03:00:00|
|2022-01-01 04:00:00|
+-------------------+
only showing top 5 rows



In [48]:
date_range_t.printSchema()

root
 |-- ds: timestamp (nullable = false)



In [49]:
#check
date_range_t.toPandas().set_index('ds').index.is_monotonic_increasing

True

In [50]:
date_range_t.toPandas().head(5)

,ds
0,2022-01-01 00:00:00
1,2022-01-01 01:00:00
2,2022-01-01 02:00:00
3,2022-01-01 03:00:00
4,2022-01-01 04:00:00


## Функция

## 4.2. **Фильтруем и создаем бакеты**

In [51]:
df_area = df.filter(df.pickup_community_area == AREA).groupby(F.date_trunc('hour', F.col('trip_start_timestamp')).alias('start_date_hour')) \
                                                     .agg(F.count('*').alias('y')) \
                                                     .orderBy('start_date_hour')

#df_area = df_area.withColumn('id', F.monotonically_increasing_id())

In [52]:
#Join 2 DF range and main
df_area = date_range_t.join(df_area, date_range_t.ds == df_area.start_date_hour, "left")

In [53]:
#Add features day_of_week, hour, month, week_of_month
df_area = df_area.withColumn("day_of_week",F.dayofweek(F.col("ds")))
df_area = df_area.withColumn("hour",F.hour(F.col("ds")))
df_area = df_area.withColumn("month",F.month(F.col("ds")))
df_area = df_area.withColumn("week_of_month", F.date_format(F.col("ds"), "W"))

In [54]:
#Fill NA Values
df_area = df_area.fillna(0, subset=['y'])

In [55]:
#Check
df_area.filter(F.col('y').isNull()).show()

+---+---------------+---+-----------+----+-----+-------------+
| ds|start_date_hour|  y|day_of_week|hour|month|week_of_month|
+---+---------------+---+-----------+----+-----+-------------+
+---+---------------+---+-----------+----+-----+-------------+



In [56]:
#Add Holidas features
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='2022-01-01', end='2024-01-01').to_pydatetime()
holidays = set([str(datetime.datetime.date(i)) for i in holidays])
holidayUDF = F.udf(lambda x: 1 if str(datetime.datetime.date(x)) in holidays else 0, ShortType())

In [57]:
df_area = df_area.withColumn('holiday', holidayUDF(F.col('ds')))

In [58]:
#Add After Holiday feature
# w = Window().partitionBy().orderBy('start_date_hour_t')
# df_area = df_area.withColumn("after_holiday", F.lag(F.col('holiday'), 1).over(w).cast("short"))

In [59]:
#Add Lags
w = Window().partitionBy().orderBy('ds')

for i in range(1, 24 + 1):
    df_area = df_area.withColumn(f"lag_{i}", F.lag(F.col('y'), i).over(w).cast("integer"))
    
    
w = Window().partitionBy().orderBy('ds')

for i in range(24, 24*7 + 2, 24):
    df_area = df_area.withColumn(f"lag_{i}", F.lag(F.col('y'), i).over(w).cast("integer"))

In [60]:
#Add sin_cos
#df_area['sin_hour'] = np.sin(2*np.pi*df['hour']/24)
#df_area['cos_hour'] = np.cos(2*np.pi*df['hour']/24)

In [61]:
#Dro NA Columns
df_area = df_area.drop('start_date_hour')
df_area = df_area.na.drop()

In [62]:
df_area.limit(5).toPandas()

,ds,y,day_of_week,hour,month,week_of_month,holiday,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,lag_48,lag_72,lag_96,lag_120,lag_144,lag_168
0,2022-01-08 00:00:00,12,7,0,1,2,0,30,35,29,46,83,102,155,117,115,82,71,71,52,60,61,77,54,18,5,1,0,5,2,11,4,7,7,3,10,37
1,2022-01-08 01:00:00,6,7,1,1,2,0,12,30,35,29,46,83,102,155,117,115,82,71,71,52,60,61,77,54,18,5,1,0,5,2,2,2,5,7,4,70
2,2022-01-08 02:00:00,11,7,2,1,2,0,6,12,30,35,29,46,83,102,155,117,115,82,71,71,52,60,61,77,54,18,5,1,0,5,2,1,5,4,2,67
3,2022-01-08 03:00:00,3,7,3,1,2,0,11,6,12,30,35,29,46,83,102,155,117,115,82,71,71,52,60,61,77,54,18,5,1,0,1,3,4,3,4,39
4,2022-01-08 04:00:00,7,7,4,1,2,0,3,11,6,12,30,35,29,46,83,102,155,117,115,82,71,71,52,60,61,77,54,18,5,1,2,3,1,6,8,26


### 5.1. Change data type

In [63]:
def change_data_type(df_t):
    """
    
    """
    df_t = df_t.withColumn("y",F.col("y").cast("integer"))
    df_t = df_t.withColumn("day_of_week",F.col("day_of_week").cast("integer"))
    df_t = df_t.withColumn("hour",F.col("hour").cast("integer"))
    df_t = df_t.withColumn("month",F.col("month").cast("integer"))
    df_t = df_t.withColumn("week_of_month",F.col("week_of_month").cast("integer"))
    df_t = df_t.withColumn("holiday",F.col("holiday").cast("integer"))

    return df_t

In [64]:
df_area = change_data_type(df_area)

In [65]:
df_area.printSchema()

root
 |-- ds: timestamp (nullable = false)
 |-- y: integer (nullable = true)
 |-- day_of_week: integer (nullable = false)
 |-- hour: integer (nullable = false)
 |-- month: integer (nullable = false)
 |-- week_of_month: integer (nullable = true)
 |-- holiday: integer (nullable = true)
 |-- lag_1: integer (nullable = true)
 |-- lag_2: integer (nullable = true)
 |-- lag_3: integer (nullable = true)
 |-- lag_4: integer (nullable = true)
 |-- lag_5: integer (nullable = true)
 |-- lag_6: integer (nullable = true)
 |-- lag_7: integer (nullable = true)
 |-- lag_8: integer (nullable = true)
 |-- lag_9: integer (nullable = true)
 |-- lag_10: integer (nullable = true)
 |-- lag_11: integer (nullable = true)
 |-- lag_12: integer (nullable = true)
 |-- lag_13: integer (nullable = true)
 |-- lag_14: integer (nullable = true)
 |-- lag_15: integer (nullable = true)
 |-- lag_16: integer (nullable = true)
 |-- lag_17: integer (nullable = true)
 |-- lag_18: integer (nullable = true)
 |-- lag_19: integer (

------

# Models

## 1. Prothet

In [66]:
# train.select(F.col("y"),F.col("ds"), F.col("pickup_community_area")).show()

In [67]:
# # Define a schema
# schema = StructType([
#                     StructField('ds', TimestampType()),
#                     StructField('y', IntegerType()),
#                     StructField('day_of_week', IntegerType()),
#                     StructField('hour', IntegerType()),
#                     StructField('pickup_community_area', IntegerType()),
#                     StructField('yhat', DoubleType()),
#                     StructField('yhat_upper', DoubleType()),
#                     StructField('yhat_lower', DoubleType()),
#                     StructField('month', IntegerType()),
#                     StructField('week_of_month', IntegerType()),
#                     StructField('holiday', IntegerType()),
#                     StructField('lag_1', IntegerType()),
#                     StructField('lag_2', IntegerType()),
#                     StructField('lag_3', IntegerType()),
#                     StructField('lag_4', IntegerType()),
#                     StructField('lag_5', IntegerType()),
#                     StructField('lag_6', IntegerType()),
#                     StructField('lag_7', IntegerType()),
#                     StructField('lag_8', IntegerType()),
#                     StructField('lag_9', IntegerType()),
#                     StructField('lag_10', IntegerType()),
#                     StructField('lag_11', IntegerType()),
#                     StructField('lag_12', IntegerType()),
#                     StructField('lag_13', IntegerType()),
#                     StructField('lag_14', IntegerType()),
#                     StructField('lag_15', IntegerType()),
#                     StructField('lag_16', IntegerType()),
#                     StructField('lag_17', IntegerType()),
#                     StructField('lag_18', IntegerType()),
#                     StructField('lag_19', IntegerType()),
#                     StructField('lag_20', IntegerType()),
#                     StructField('lag_21', IntegerType()),
#                     StructField('lag_22', IntegerType()),
#                     StructField('lag_23', IntegerType()),
#                     StructField('lag_24', IntegerType()),
#                     StructField('lag_48', IntegerType()),
#                     StructField('lag_72', IntegerType()),
#                     StructField('lag_96', IntegerType()),
#                     StructField('lag_120', IntegerType()),
#                     StructField('lag_144', IntegerType()),
#                     StructField('lag_168', IntegerType()),
# ])

In [68]:
# # define the Pandas UDF
# @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
# def apply_model(df_in: pd.DataFrame):
#     """
    
#     """
#     model = Prophet(
#       interval_width=0.95,
#       daily_seasonality=True,
#       weekly_seasonality=True,
#       yearly_seasonality=True,
#       seasonality_mode='multiplicative'
#     )
#     # fit the model to historical dat
#     model.add_country_holidays(country_name='US')
    
#     model.fit(df_in)
    
#     # Create a data frame that lists 7 days
#     future = model.make_future_dataframe(
#       periods=0,
#       freq='H',
#       include_history=True)
    
#     # Out of sample prediction
#     future = model.predict(future)
    
#     # Create a data frame that contains store, item, y, and yhat
#     f_pd = future[['ds', 'yhat', 'yhat_upper', 'yhat_lower']]
#     st_pd = df_in[['ds', 'y', '']]
#     result_pd = f_pd.join(st_pd.set_index('ds'), on='ds', how='left')

#     result_pd['yhat'] = result_pd['yhat'].clip(lower=0)
#     result_pd['yhat_lower'] = result_pd['yhat_lower'].clip(lower=0)
#     result_pd['yhat_upper'] = result_pd['yhat_upper'].clip(lower=0)
    
#     return result_pd

In [69]:
# train.columns

In [70]:
# # Apply the function to all store-items
# results = train.groupby(['pickup_community_area']).apply(apply_model)

In [71]:
# results.show()

In [72]:
# # Apply the function to all store-items
# results = train.groupby(['pickup_community_area']).apply(apply_model)
# t = results.toPandas().set_index('ds')

In [73]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=t.index, y=t.y,
#                     mode='lines',
#                     name='y'))
# fig.add_trace(go.Scatter(x=t.index, y=t.yhat,
#                     mode='lines',
#                     name='yhat'))
# fig.show()

## 6. Train Test Split

In [74]:
feature_list = list(df_area.columns)
feature_list.remove('y')
feature_list.remove('ds')
feature_list

['day_of_week',
 'hour',
 'month',
 'week_of_month',
 'holiday',
 'lag_1',
 'lag_2',
 'lag_3',
 'lag_4',
 'lag_5',
 'lag_6',
 'lag_7',
 'lag_8',
 'lag_9',
 'lag_10',
 'lag_11',
 'lag_12',
 'lag_13',
 'lag_14',
 'lag_15',
 'lag_16',
 'lag_17',
 'lag_18',
 'lag_19',
 'lag_20',
 'lag_21',
 'lag_22',
 'lag_23',
 'lag_24',
 'lag_48',
 'lag_72',
 'lag_96',
 'lag_120',
 'lag_144',
 'lag_168']

In [75]:
featureassembler = VectorAssembler(inputCols=feature_list,
                                   outputCol="independent")

In [76]:
output = featureassembler.transform(df_area)

In [77]:
#feature Extraction
final_data = output.select("independent", "y", "ds")
final_data.show()

+--------------------+---+-------------------+
|         independent|  y|                 ds|
+--------------------+---+-------------------+
|[7.0,0.0,1.0,2.0,...| 12|2022-01-08 00:00:00|
|[7.0,1.0,1.0,2.0,...|  6|2022-01-08 01:00:00|
|[7.0,2.0,1.0,2.0,...| 11|2022-01-08 02:00:00|
|[7.0,3.0,1.0,2.0,...|  3|2022-01-08 03:00:00|
|[7.0,4.0,1.0,2.0,...|  7|2022-01-08 04:00:00|
|[7.0,5.0,1.0,2.0,...|  5|2022-01-08 05:00:00|
|[7.0,6.0,1.0,2.0,...|  7|2022-01-08 06:00:00|
|[7.0,7.0,1.0,2.0,...| 18|2022-01-08 07:00:00|
|[7.0,8.0,1.0,2.0,...| 25|2022-01-08 08:00:00|
|[7.0,9.0,1.0,2.0,...| 39|2022-01-08 09:00:00|
|[7.0,10.0,1.0,2.0...| 36|2022-01-08 10:00:00|
|[7.0,11.0,1.0,2.0...| 48|2022-01-08 11:00:00|
|[7.0,12.0,1.0,2.0...| 51|2022-01-08 12:00:00|
|[7.0,13.0,1.0,2.0...| 46|2022-01-08 13:00:00|
|[7.0,14.0,1.0,2.0...| 61|2022-01-08 14:00:00|
|[7.0,15.0,1.0,2.0...| 59|2022-01-08 15:00:00|
|[7.0,16.0,1.0,2.0...| 69|2022-01-08 16:00:00|
|[7.0,17.0,1.0,2.0...| 71|2022-01-08 17:00:00|
|[7.0,18.0,1.

# 4. Train Test Split


In [85]:
def train_val_test_split(df_t):
    """
    
    """
    df_t = df_t.withColumn("rank", F.percent_rank().over(Window.partitionBy().orderBy("ds")))
    #df_t = df_t.withColumn('pickup_community_area', F.lit(area))
    train = df_t.filter(F.col("rank") <= 0.8).drop('rank')
    val = df_t.filter(F.col("rank") > 0.8).drop('rank')
   # test = df_t.filter(F.col("rank") > 0.9).drop('rank')
    test = val.tail(1)

    return train, val

In [86]:
train, val = train_val_test_split(final_data)

In [87]:
(train.count(), len(train.columns)), (val.count(), len(val.columns))

((6873, 3), (1719, 3))

# LinearRegression

In [88]:
model_l_reg = LinearRegression(featuresCol='independent',  maxIter = 100, 
                               labelCol='y', regParam = 0.4, elasticNetParam = 0.1)

In [89]:
model_l_reg = model_l_reg.fit(train)

In [90]:
model_l_reg.coefficients, model_l_reg.intercept

(DenseVector([-0.9575, -0.2602, -0.4285, -0.0972, -8.9595, 0.7212, -0.0997, -0.0625, 0.0131, 0.0041, -0.023, 0.0072, -0.0009, 0.0376, 0.0135, -0.0637, -0.0523, 0.0259, 0.0946, 0.0057, -0.0611, -0.0071, 0.0177, 0.0084, 0.0052, -0.0107, 0.0492, 0.1589, 0.0949, -0.1155, 0.0253, -0.0025, -0.0085, 0.0874, 0.1701]),
 6.398768292490147)

In [91]:
pred = model_l_reg.evaluate(val)
pred.predictions.show()

+--------------------+---+-------------------+------------------+
|         independent|  y|                 ds|        prediction|
+--------------------+---+-------------------+------------------+
|[6.0,9.0,10.0,4.0...|194|2022-10-21 09:00:00|208.30123564269525|
|[6.0,10.0,10.0,4....|147|2022-10-21 10:00:00| 211.3140177301509|
|[6.0,11.0,10.0,4....|192|2022-10-21 11:00:00| 183.3710317960467|
|[6.0,12.0,10.0,4....|186|2022-10-21 12:00:00| 218.2028902776608|
|[6.0,13.0,10.0,4....|208|2022-10-21 13:00:00|211.99620521126815|
|[6.0,14.0,10.0,4....|235|2022-10-21 14:00:00| 236.5262957318217|
|[6.0,15.0,10.0,4....|245|2022-10-21 15:00:00|260.09957823005874|
|[6.0,16.0,10.0,4....|319|2022-10-21 16:00:00|269.30763229042594|
|[6.0,17.0,10.0,4....|316|2022-10-21 17:00:00|  312.108783921869|
|[6.0,18.0,10.0,4....|306|2022-10-21 18:00:00|279.40218839808534|
|[6.0,19.0,10.0,4....|246|2022-10-21 19:00:00|254.98823245057167|
|[6.0,20.0,10.0,4....|139|2022-10-21 20:00:00|183.26200639609513|
|[6.0,21.0

In [92]:
pred = model_l_reg.evaluate(train)
pred.predictions.show()

+--------------------+---+-------------------+------------------+
|         independent|  y|                 ds|        prediction|
+--------------------+---+-------------------+------------------+
|[7.0,0.0,1.0,2.0,...| 12|2022-01-08 00:00:00|23.315630866220925|
|[7.0,1.0,1.0,2.0,...|  6|2022-01-08 01:00:00|15.508344966375152|
|[7.0,2.0,1.0,2.0,...| 11|2022-01-08 02:00:00|14.093310543397875|
|[7.0,3.0,1.0,2.0,...|  3|2022-01-08 03:00:00|12.248381301170657|
|[7.0,4.0,1.0,2.0,...|  7|2022-01-08 04:00:00|1.4021051093740935|
|[7.0,5.0,1.0,2.0,...|  5|2022-01-08 05:00:00| 8.828789631784431|
|[7.0,6.0,1.0,2.0,...|  7|2022-01-08 06:00:00|19.309421469909182|
|[7.0,7.0,1.0,2.0,...| 18|2022-01-08 07:00:00|26.656102898167514|
|[7.0,8.0,1.0,2.0,...| 25|2022-01-08 08:00:00|28.378746049334094|
|[7.0,9.0,1.0,2.0,...| 39|2022-01-08 09:00:00|28.688453249797618|
|[7.0,10.0,1.0,2.0...| 36|2022-01-08 10:00:00| 38.84380168612079|
|[7.0,11.0,1.0,2.0...| 48|2022-01-08 11:00:00| 43.38139180501331|
|[7.0,12.0

In [100]:
print(pred.meanAbsoluteError)

15.42234591969822


### RandomForestRegressor

In [111]:
rfr = RandomForestRegressor(featuresCol="independent",
                            labelCol="y",
                            maxDepth = 12,
                            subsamplingRate = 0.8)
model = rfr.fit(train)

In [113]:
predict = model.transform(val)
predict.show()

+--------------------+---+-------------------+------------------+
|         independent|  y|                 ds|        prediction|
+--------------------+---+-------------------+------------------+
|[6.0,9.0,10.0,4.0...|194|2022-10-21 09:00:00|191.47684761610606|
|[6.0,10.0,10.0,4....|147|2022-10-21 10:00:00|213.87761670024616|
|[6.0,11.0,10.0,4....|192|2022-10-21 11:00:00| 208.2281634233287|
|[6.0,12.0,10.0,4....|186|2022-10-21 12:00:00| 206.5069048256596|
|[6.0,13.0,10.0,4....|208|2022-10-21 13:00:00|200.68835580605176|
|[6.0,14.0,10.0,4....|235|2022-10-21 14:00:00|235.65344525105866|
|[6.0,15.0,10.0,4....|245|2022-10-21 15:00:00| 275.1048015873016|
|[6.0,16.0,10.0,4....|319|2022-10-21 16:00:00|  292.047823170981|
|[6.0,17.0,10.0,4....|316|2022-10-21 17:00:00| 324.4505173472361|
|[6.0,18.0,10.0,4....|306|2022-10-21 18:00:00| 302.8371962707105|
|[6.0,19.0,10.0,4....|246|2022-10-21 19:00:00| 254.4748622211123|
|[6.0,20.0,10.0,4....|139|2022-10-21 20:00:00|161.56355519480522|
|[6.0,21.0

# Spark XGBRegressor

In [162]:
SparkXGBClassifier_def = SparkXGBRegressor(max_depth=6,
                                  features_col='independent',
                                  label_col='y')

In [163]:
classifier_model = SparkXGBClassifier_def.fit(train)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[15:52:14] task 0 got new rank 0                                    (0 + 1) / 1]
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:782: UserWarning:    

Loading a native XGBoost model with Scikit-Learn interface.



In [164]:
predict = classifier_model.transform(val)
predict.show()

+--------------------+---+-------------------+------------------+
|         independent|  y|                 ds|        prediction|
+--------------------+---+-------------------+------------------+
|[6.0,9.0,10.0,4.0...|194|2022-10-21 09:00:00|186.58364868164062|
|[6.0,10.0,10.0,4....|147|2022-10-21 10:00:00|190.35731506347656|
|[6.0,11.0,10.0,4....|192|2022-10-21 11:00:00| 198.9430389404297|
|[6.0,12.0,10.0,4....|186|2022-10-21 12:00:00|201.50718688964844|
|[6.0,13.0,10.0,4....|208|2022-10-21 13:00:00|187.03775024414062|
|[6.0,14.0,10.0,4....|235|2022-10-21 14:00:00| 225.1385040283203|
|[6.0,15.0,10.0,4....|245|2022-10-21 15:00:00| 285.8316345214844|
|[6.0,16.0,10.0,4....|319|2022-10-21 16:00:00| 295.8230285644531|
|[6.0,17.0,10.0,4....|316|2022-10-21 17:00:00| 329.1588134765625|
|[6.0,18.0,10.0,4....|306|2022-10-21 18:00:00| 306.3639831542969|
|[6.0,19.0,10.0,4....|246|2022-10-21 19:00:00|238.42369079589844|
|[6.0,20.0,10.0,4....|139|2022-10-21 20:00:00|153.53456115722656|
|[6.0,21.0

In [189]:
def mape(y_true, y_pred):
    mape = mean_absolute_percentage_error(np.array([y_true]), np.array([y_pred]))
    return float(mape)

mape_cols = F.udf(mape, DoubleType())


def mae(y_true, y_pred):
    mape = mean_absolute_error(np.array([y_true]), np.array([y_pred]))
    return float(mape)

mae_cols = F.udf(mae, DoubleType())

In [190]:
predict = predict.withColumn('mape', mape_cols('y', 'prediction'))
predict = predict.withColumn('mae', mae_cols('y', 'prediction'))

In [191]:
predict.show()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


+--------------------+---+-------------------+------------------+--------------------+------------------+
|         independent|  y|                 ds|        prediction|                mape|               mae|
+--------------------+---+-------------------+------------------+--------------------+------------------+
|[6.0,9.0,10.0,4.0...|194|2022-10-21 09:00:00|186.58364868164062| 0.03822861504308956| 7.416351318359375|
|[6.0,10.0,10.0,4....|147|2022-10-21 10:00:00|190.35731506347656| 0.29494772152024873| 43.35731506347656|
|[6.0,11.0,10.0,4....|192|2022-10-21 11:00:00| 198.9430389404297| 0.03616166114807129|6.9430389404296875|
|[6.0,12.0,10.0,4....|186|2022-10-21 12:00:00|201.50718688964844|  0.0833719725249916|15.507186889648438|
|[6.0,13.0,10.0,4....|208|2022-10-21 13:00:00|187.03775024414062| 0.10078004690317008|20.962249755859375|
|[6.0,14.0,10.0,4....|235|2022-10-21 14:00:00| 225.1385040283203| 0.04196381264544548| 9.861495971679688|
|[6.0,15.0,10.0,4....|245|2022-10-21 15:00:00|

In [192]:
t = predict.toPandas().set_index('ds')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [209]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t.index, y=t.y,
                    mode='lines',
                    name='y',
                    line=dict(color='blue', width=2,
                              )
                    ))
fig.add_trace(go.Scatter(x=t.index, y=t.prediction,
                    mode='lines',
                    name='prediction',
                    line=dict(color='red', width=2, dash='dot')
                    ))
fig.show()

In [180]:
predict.limit(300).select(F.mean('mape')).show()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


+-------------------+
|          avg(mape)|
+-------------------+
|0.24803440082581582|
+-------------------+



In [178]:
300/24

12.5

In [ ]:

a=spark.createDataFrame([(101, 1, 16)], ['ID', 'A', 'B'])
a.show()
a.withColumn('Result', sum_cols('A', 'B')).show()

In [ ]:
a.withColumn('Result', sum_cols('A', 'B')).show()

### MAPE

In [ ]:
@pandas_udf(final_schema, PandasUDFType.GROUPED_MAP)
def gr_mape_val(join_df):
    
    mape = mean_absolute_percentage_error(join_df["y"], join_df["yhat"]) 
    join_df['mape'] = mape
  
  return join_df

df_apply = join_df.groupby('ID').applyInPandas(gr_mape_val, final_schema)
df_apply.show()

## 6. XGBoost

In [ ]:
# from pyspark.sql.functions import when, lit, col
# df3 = df2.withColumn("lit_value2", when((col("Salary") >=40000) & (col("Salary") <= 50000),lit("100")).otherwise(lit("200")))
# df3.show(truncate=False)